# <font color=red>Note</font>
- Function
    - compute count aggregation function for string attributes (to get how many records)
        - So, this code works for aggregation function like <font color=blue> "x_count_prod"</font>
    - compute avg, sum, count for numeric attributes 
- For indexing part, it is based on the [C] rather than [V]. 
    - <font color=red> when swap the "prod" and "cust", the [C] part should also indexable </font>
    - Therefore the MF_map construction has been changed and fix the bug 

## sample 7_2 swap error explaination
SELECT ATTRIBUTE(S):

<font color=red>prod, cust, </font>x_sum_quant,  <font color=blue>x_count_prod,</font>  x_avg_quant+1, y_avg_quant/2 ,x_avg_quant/y_avg_quant

NUMBER OF GROUPING VARIABLES(n):

x,y

GROUPING ATTRIBUTES(V):

prod, cust

F-VECT([F]):

x_avg_quant, y_avg_quant, x_count_prod, x_sum_quant

SELECT CONDITION-VECT([C]):

<font color=red>x.cust=cust and x.prod=prod </font> <font color = green>(not consistent with S will make index error)</font>

y.cust=cust and y.prod<>prod  

HAVING_CONDITION(G):

y_avg_quant<>0 and x_avg_quant>=0

## sample 7_2  This will give right result with swap [C] as well
SELECT ATTRIBUTE(S):

<font color = green>prod, cust,</font> x_sum_quant, x_count_prod,  x_avg_quant+1, y_avg_quant/2 ,x_avg_quant/y_avg_quant

NUMBER OF GROUPING VARIABLES(n):

x,y

GROUPING ATTRIBUTES(V):

prod, cust

F-VECT([F]):

x_avg_quant, y_avg_quant, x_count_prod, x_sum_quant

SELECT CONDITION-VECT([C]):

<font color = green> x.prod=prod and x.cust=cust</font>

<font color = green> y.prod<>prod and y.cust=cust</font>

HAVING_CONDITION(G):

y_avg_quant<>0 and x_avg_quant>=0

## <font color=red> Correction the code after presentation</font>
- change the MF_MAP index building method 
- Add the reverse combination indexable as well
      t += " + ".join(['str(tmprec' + str([j]) + ")" for j in list(idx_combination)[::-1]]) 
      t += " + ".join(['str(tmprec' + str([j]) + ")" for j in list(idx_combination)]) 

- Result: those two combination will be indexable as well
      MFMap[str(tmprec['prod']) + str(tmprec['cust'])]=MF_idx
      MFMap[str(tmprec['cust']) + str(tmprec['prod'])]=MF_idx



## sample 1 My code can handle this. 
- the result in cols 3_avg_quant, 3_sum_quant are the same because <font color = red>  there is only one record for gv = 3</font>

SELECT ATTRIBUTE(S):

cust,    1_avg_quant, 2_avg_quant,   3_avg_quant, 3_sum_quant

NUMBER OF GROUPING VARIABLES(n):

1,2,3

GROUPING ATTRIBUTES(V):

cust

F-VECT([F]):

1_sum_quant,    1_avg_quant, 2_avg_quant, 3_sum_quant, 3_avg_quant, 3_sum_quant

SELECT CONDITION-VECT([C]):

1.cust=cust     and 1.state='NY' and 1.year=2005

2.cust=cust and      2.state='CT' and 2.year=2005

3.cust=cust and 3.state='NJ' and 3.year=2005

HAVING_CONDITION(G):

1_avg_quant > 2_avg_quant and 1_avg_quant >= 3_avg_quant

# main Function

In [108]:
from collections import defaultdict
import psycopg2
from emf2py import emf_to_py
from utils import *
import argparse
import re
import sys
# 初始化数据库 默认读取参数





## Set input file in there

In [ ]:
output_num = '7_2'

input_file="input"
filename = "output"
input_path = "E:/OneDrive - stevens.edu/Stevens BIA/CS562/Proj/EMF2PY/Input_files/"
input_file = "input%s.txt"% output_num


## Connect DB

In [109]:
########################################
# connect DB login  ==> get schema & log info
########################################
table = "sales"
dbname = "postgres"
host = "localhost"
user = "postgres"
password = "00000000"
port = "5388"

try:
	connection = psycopg2.connect(user=user, password=password, host=host, port=port, database=dbname)
	query = "SELECT column_name, data_type FROM information_schema.columns WHERE table_name = '" + table + "';"
	cursor = connection.cursor()
	cursor.execute(query)
	if cursor.rowcount == 0:
		print("Connected, but no data in table: %s " % table)
		sys.exit(0)
	schema = cursor.fetchall()
except(Exception, psycopg2.Error) as error:
	print("Error connecting to PostgreSQL database ==>", error)
	sys.exit(0)

print("=" * 40)
print("In <<'%s'>> table" % format(table))
print("=" * 40)
print("ATTRIBUTES".ljust(14, " "), "DTYPE".ljust(15, " "))
for i in schema:
	print("*", i[0].ljust(12, " "), i[1].ljust(15, " "))
print("=" * 40)



In <<'sales'>> table
ATTRIBUTES     DTYPE          
* cust         character varying
* prod         character varying
* day          integer        
* month        integer        
* year         integer        
* state        character      
* quant        integer        


## Get schema & log info
- ready to pass to emf2py

In [110]:
PG_log_info = {'user':user, 'password':password,'host':host, 'port':port, 'database':dbname}
initial_list=['user', 'password', 'host', 'port', 'database']

## Utils - Parsing input file

In [111]:
########################################
### Parsing for Input txt
########################################
import re

def txt_parsing(txt):
    # build the map for input text
    key_place = {}
    keyword = ["(S)", "(n)", "(V)", "([F])", "([C])", "(G)"]
    for i in range(len(keyword)):
        for j in range(i, len(txt)):
            if txt[j].find(keyword[i]) > 0:
                key_place[keyword[i]] = j
    # {'(S)': 0, '(n)': 2, '(V)': 4, '([F])': 6, '([C])': 8, '(G)': 12}
    # print(key_place)
    
    # find the distance with next key word
    params = key_place
    tmp = list(key_place.keys())  # ['(S)', '(n)', '(V)', '([F])', '([σ])', '(G)']
    for i in range(len(key_place) - 1):
        begin = key_place[tmp[i]] + 1  # begin with '(S)': 0+1
        end = key_place[tmp[i + 1]]  # end with '(n)': 2
        #     print(key_place[tmp[i]]+1,key_place[tmp[i+1]])
        params[keyword[i]] = txt[begin:end]  # find the text from begin to end
    # print(txt[key_place[tmp[i+1]]+1:]) # the last one
    params[keyword[i + 1]] = txt[key_place[tmp[i + 1]] + 1:]
    
    # params
    # '''
    # {'(S)': ['cust,1_avg_quant, 2_avg_quant,3_avg_quant'],
    #  '(n)': ['1,2,3'],
    #  '(V)': ['cust'],
    #  '([F])': ['1_sum_quant, 1_avg_quant, 2_avg_quant, 3_sum_quant, 3_avg_quant'],
    #  '([C])': ["1.cust=cust and 1.state='NY' and 1.year=2005", "2.cust=cust and 2.state='CT' and 2.year=2005", "3.cust=cust and 3.state='NJ' and 3.year=2005"],
    #  '(G)': ['1_avg_quant > 2_avg_quant and 1_avg_quant > 3_avg_quant']}'''
    
    # handle several lines
    S = [i.strip() for i in params['(S)'][0].split(',')]
    n = [i.strip() for i in params['(n)'][0].split(',')]
    V = [i.strip() for i in params['(V)'][0].split(',')]
    F = [i.strip() for i in params['([F])'][0].split(',')]
    C = params["([C])"]
    G = [i.strip() for i in params['(G)'][0].split(',')]
    return S, n, V, F, C, G


with open(input_path+input_file, 'r', encoding='utf') as f:
    txt = f.read().strip().split('\n')
    
S,n,V,F,C,G = txt_parsing(txt)

In [112]:
S

['prod',
 'cust',
 'x_sum_quant',
 'x_count_prod',
 'x_avg_quant+1',
 'y_avg_quant/2',
 'x_avg_quant/y_avg_quant']

In [113]:
n

['x', 'y']

In [114]:
S

['prod',
 'cust',
 'x_sum_quant',
 'x_count_prod',
 'x_avg_quant+1',
 'y_avg_quant/2',
 'x_avg_quant/y_avg_quant']

In [115]:
n

['x', 'y']

In [116]:
V

['prod', 'cust']

In [117]:
F

['x_avg_quant', 'y_avg_quant', 'x_count_prod', 'x_sum_quant']

In [118]:
C

['x.prod=prod and x.cust=cust', 'y.prod<>prod and y.cust=cust']

In [119]:
G

['y_avg_quant<>0 and x_avg_quant>=0']

# EMF Processing 

In [120]:
from itertools import combinations_with_replacement as cmb

## initialization For variables
- T_cols = ['cust', 'prod', 'day', 'month', 'year', 'state', 'quant']
- V_cols = ['cust', 'prod']
- MF_cols = V+F+(other agg function - avg extend to sum count) 
    - = ['cust', 'prod', 'x_count_quant', 'x_sum_quant', 'x_avg_quant', 'y_count_quant', 'y_sum_quant', 'y_avg_quant', 'x_count_prod', 'x_sum_quant', 'y_count_prod']
    - <font color=red>For each gv and attrs related to agg</font>
    
- MF_dtypes = For retrieve type, Save all combination of (gv + agg + attributes). But dont need to add into MF_cols
- C_cols = ['1.prod=prod and 1.month=month and 1.year=1997', '2.prod=prod and 2.year=1997'] # condition 

In [121]:
########################################
### initialization
########################################
# T_cols = ['cust', 'prod', 'day', 'month', 'year', 'state', 'quant']
T_cols = [i[0] for i in schema]

## three parsing mapping
sql_dtypes_maps = {"character varying": "''", "character": "''", "integer": 0}
logic_map = {">=": " >= ", "<=": " <= ", "=": " == ", "<>": ' != ', ">": " > ", "<": " < "}  # for insert space ans subtitute logic symbol in order. Only execute once in loop
format_length_map = {"''": "{:<12}", 0: "{:>15}", 0.0: "{:>15}"}


V_cols = [i.strip() for i in V]  # grouping variable = ['cust', 'prod']

# MF_dtypes = {'cust': "''", 'prod': "''", 'day': 0, 'month': 0, 'year': 0, 'state': "''", 'quant': 0, '_1_avg_quant': 0.0, '_1_count_quant': 0, '_1_sum_quant': 0.0, '_1_max_quant': 0.0, '_1_min_quant': 0.0, '_2_avg_quant': 0.0, '_2_count_quant': 0, '_2_sum_quant': 0.0, '_2_max_quant': 0.0, '_2_min_quant': 0.0}
# MF_cols = ['prod', 'month', '_1_avg_quant', '_1_count_quant', '_1_sum_quant', '_1_max_quant', '_1_min_quant', '_2_avg_quant', '_2_count_quant', '_2_sum_quant', '_2_max_quant', '_2_min_quant'... month etc ]
MF_cols = V[:] # Grouping variables
for agg_col in F[:]: # for agg function. build MF_Vector
    tmp_mf = []
    if "avg" in agg_col:
        tmp_mf.append(agg_col.replace('avg', 'count'))
        tmp_mf.append(agg_col.replace('avg', 'sum'))
        tmp_mf.append(agg_col.replace('avg', 'avg'))
    else:
        tmp_mf = [agg_col]
    MF_cols.extend(tmp_mf)
    

########## save dtype for retrieve type But dont need to add into MF_cols
MF_dtypes = {i[0]: sql_dtypes_maps[i[1]] for i in schema}
for i in schema:  # ('prod', 'character varying'),...]
    if type(sql_dtypes_maps[i[1]]) == str:  # if it is string
        for gv in n:
            Agg_F = ["{}_count_{}".format(gv, i[0])]
            #             MF_cols.extend(Agg_F)
            MF_dtypes.update({i: j for i, j in zip(Agg_F, [0])})
    else:  # int
        for gv in n:
            Agg_F = ("{}_avg_{},{}_count_{},{}_sum_{},{}_max_{},{}_min_{}".format(gv, i[0], gv, i[0], gv, i[0], gv, i[0],gv, i[0])).split(',')
            #   MF_cols.extend(Agg_F) # There only focus on quant. if other than quant, there should be more
            MF_dtypes.update({i: j for i, j in zip(Agg_F, [0.0, 0, 0.0, 0.0, 0.0])})
########## 


C_cols = [row.strip().split("and") for row in C if len(row) > 2]
# ['1.prod=prod and 1.month=month and 1.year=1997', '2.prod=prod and 2.year=1997']


In [122]:
print(MF_cols)

['prod', 'cust', 'x_count_quant', 'x_sum_quant', 'x_avg_quant', 'y_count_quant', 'y_sum_quant', 'y_avg_quant', 'x_count_prod', 'x_sum_quant']


## Connection_initialize 

In [123]:
########################################
### Connection_initialize
########################################
import_text = 'import psycopg2\nimport sys\nfrom collections import defaultdict'
initial_list = list(PG_log_info.keys())
connection_text = '\ntry:\n    connection = psycopg2.connect('
for i in initial_list:
    connection_text += str(i) + "='" + str(PG_log_info[i]) + "', "
connection_text += ''')\n    query = "SELECT * FROM sales"\n    cursor = connection.cursor()\n    cursor.execute(query)\n    if cursor.rowcount == 0:\n        print("Connected, but no data!")\n        sys.exit(0)
                    \n    records = cursor.fetchall()\nexcept(Exception, psycopg2.Error) as error:\n    print("Error connecting to PostgreSQL database ==>", error)\n    sys.exit(0)\n
                    '''

print(connection_text)


try:
    connection = psycopg2.connect(user='postgres', password='00000000', host='localhost', port='5388', database='postgres', )
    query = "SELECT * FROM sales"
    cursor = connection.cursor()
    cursor.execute(query)
    if cursor.rowcount == 0:
        print("Connected, but no data!")
        sys.exit(0)
                    
    records = cursor.fetchall()
except(Exception, psycopg2.Error) as error:
    print("Error connecting to PostgreSQL database ==>", error)
    sys.exit(0)

                    


## Initializing MF_Vector

In [124]:
########################################
# Initializing MF_Vector
########################################
MFVector_text = "MFVector = defaultdict(lambda: {"
for col_name in MF_cols:
    MFVector_text += "'" + col_name + "'" + ':' + str(MF_dtypes[col_name]) + ', '
MFVector_text = MFVector_text[:-2] + '})' + '\nMF_idx=0\nMFMap = {}\n'

print(MFVector_text)

MFVector = defaultdict(lambda: {'prod':'', 'cust':'', 'x_count_quant':0, 'x_sum_quant':0.0, 'x_avg_quant':0.0, 'y_count_quant':0, 'y_sum_quant':0.0, 'y_avg_quant':0.0, 'x_count_prod':0, 'x_sum_quant':0.0})
MF_idx=0
MFMap = {}



## Initializing Map for MF_Structure

In [125]:
########################################
# Initializing Map for MF_Structure
########################################
scan_text = "for i in range(len(records)):\n\ttmprec = {x[0]:x[1] for x in zip(" + str(T_cols) + ",records[i])}\n"

MFMap_initing = []
MFMap_initing.append("\tif (" + " + ".join(['str(tmprec' + str([i]) + ")" for i in V_cols]) + ") not in MFMap:\n")
MFMap_initing.append( "\t\t" + "\n\t\t".join(['MFVector[MF_idx]' + str([i]) + '=tmprec' + str([i]) for i in V_cols]) + '\n')

for idx_combination in cmb(V_cols, len(V_cols)):  # ['cust', 'prod] ==>>> ['cust','cust],['prod','prod'], ['cust','prod']
    t = '\t\tMFMap['
    if len(set(idx_combination)) > 1:  # if they are not the same
        t = '\t\tMFMap['
        t += " + ".join(['str(tmprec' + str([j]) + ")" for j in list(idx_combination)])  # MFMap[tmprec['prod'] + tmprec['month']]=MF_idx
        t += ']=MF_idx\n'
        
        ######################################## Correction on May 8
        t += '\t\tMFMap['
        t += " + ".join(['str(tmprec' + str([j]) + ")" for j in list(idx_combination)[::-1]]) 
        t += ']=MF_idx\n'
        #########################################
    else:  # if they are the same
        t += 'str(tmprec' + str(list(set(idx_combination))) + ")"  # MFMap[tmprec['prod']]=MF_idx
        t += ']=MF_idx\n'
    MFMap_initing.append(t)

MFMap_text = scan_text + "".join(MFMap_initing) + "\t\tMF_idx+=1 # new idx\n\n"  # MFMap_initial_text

print(MFMap_text)

for i in range(len(records)):
	tmprec = {x[0]:x[1] for x in zip(['cust', 'prod', 'day', 'month', 'year', 'state', 'quant'],records[i])}
	if (str(tmprec['prod']) + str(tmprec['cust'])) not in MFMap:
		MFVector[MF_idx]['prod']=tmprec['prod']
		MFVector[MF_idx]['cust']=tmprec['cust']
		MFMap[str(tmprec['prod'])]=MF_idx
		MFMap[str(tmprec['prod']) + str(tmprec['cust'])]=MF_idx
		MFMap[str(tmprec['cust']) + str(tmprec['prod'])]=MF_idx
		MFMap[str(tmprec['cust'])]=MF_idx
		MF_idx+=1 # new idx




In [126]:
list(idx_combination)

['cust', 'cust']

## <font color=red>Updating for MF_Structure  </font>

In [127]:

########################################
## # Updating for MF_Structure  -  V1
########################################
MF_update = "for gv in " + str(n) + ":\n"
scan_text_2 = "\t\tfor i in range(len(records)):\n\t\t\ttmprec = {x[0]:x[1] for x in zip(" + str(T_cols) + ",records[i])}\n"
for one_gv in range(len(n)):
    Map_index = '\t\t\tindex = MFMap['
    indexing_cols = []
    Condition = '\t\t\tif ('
    conditioning_cols = []
    Update = ''
    updating_cols = []
    for piece in C_cols[one_gv]:  # C_cols = [['1.prod=prod ', ' 1.month=month ', ' 1.year=1997'], ['2.prod=prod ', ' 2.year=1997']]

        ############# indexing_cols  E.g. 1.cust=cust
        #### output ####  index = MFMap[tmprec['prod']+tmprec['month']]
        if ('=' in piece) and (">=" not in piece) and ("<=" not in piece):  # find the text after "="
            t = piece.split("=")[-1].strip()  # find the indexing cols which exist "=" symbol. E.g. 1.cust=cust , the later 'cust'
            # print(t)
            if t in V_cols:  # intersect with MF_Vector combined index
                # print(V_cols)
                indexing_cols.append("str(tmprec['" + t + "'])")  # This have to be string
                t = ''

        ############ Condition  # subsitute the condition symbol in the map
        #### output #### if ((tmprec['prod']==MFVector[index]['prod']) and (tmprec['month']==MFVector[index]['month']) and (tmprec['year']==1997)):
        for key in logic_map.keys():  # order in the logic map matter. if find the '=' first,  the">=" maybe include
            if key in piece:  # subsitute, and break into two part, based on " " whitespace because the map contains the white space in both side
                #                 x = piece.replace(key, logic_map[key]).strip().split(" ")
                x = [i.strip() for i in piece.replace(key, logic_map[key]).split(" ") if len(i.strip()) >= 1]  # '2.prod=prod' ==> [2.prod, ==,  prod]
                break  # only replace once

        if x[2] in V_cols:  # the later one not exist in V_cols for MF_V indexing
            conditioning_cols.append("(tmprec['" + x[0][(x[0].find('.') + 1):] + "']" + x[1] + "MFVector[index][" + "'" + x[2] + "'])")  # x =  [2.prod, ==,  prod]
        else:
            conditioning_cols.append("(tmprec['" + x[0][(x[0].find('.') + 1):] + "']" + x[1] + x[2] + ")")  # E.g. x = [3.year, ==, 1997]

    ############ Update loop for each gv
    agg_cols = [col for col in MF_cols if (n[one_gv]) == col[0]]  # find the related gv cols in F. at the first begining one . e.g. gv = first of '2_avg_quant'
    #     print(agg_cols)

    ######## key code: for handle one gv for other attrs
    ##### e.g. for [x_count_prod,  x_avg_quant, x_sum_quant]
    ##### get gv_attrs_set = [prod, quant]
    ########
    gv_attrs_set = list(set([gv_attrs.split('_')[2] for gv_attrs in agg_cols]))  # get gv_attrs_set = [prod, quant]
    for attrs in gv_attrs_set:  # for gv=x, for attrs in [prod, quant] ...
        agg_attr_cols = [each for each in agg_cols if each.find(attrs) > 1]  # 找到该 gv 下面 只包含  prod 的所有 agg_cols =  x_count_prod

        updating_col = []  # one gv, one attrs for it
        for agg_col in agg_attr_cols:
            y = agg_col.split('_')  # ['2', 'avg', 'quant'] # meet avg , break
            # identify agg function
            if 'avg' == y[-2]:
                updating_col = []  # meet 'avg', append with sum and count
                updating_col.append("\t\t\t\tMFVector[index]['" + agg_col.replace('avg', 'count') + "']+=1\n")
                updating_col.append( "\t\t\t\tMFVector[index]['" + agg_col.replace('avg', 'sum') + "']+=" + "tmprec['" + y[-1] + "']\n")
                updating_col.append("\t\t\t\tMFVector[index]['" + agg_col + "']=" + "MFVector[index]['" + agg_col.replace('avg','sum') + "']" 
                                    + '/' + "MFVector[index]['" + agg_col.replace('avg', 'count') + "']\n")
                break
            elif ('sum' == y[-2]):
                updating_col.append("\t\t\t\tMFVector[index]['" + agg_col.replace('avg', 'sum') + "']+=" + "tmprec['" + y[-1] + "']\n")
            elif ('count' == y[-2]):
                updating_col.append("\t\t\t\tMFVector[index]['" + agg_col.replace('avg', 'count') + "']+=1\n")
            else:
                print(":< This poor codes cannot handle this aggregation function:", y[-2])
        #             print(updating_col) # one gv, one attrs for it
        updating_cols.extend(updating_col)  # one gv, every attrs for it!!
    #         print(updating_cols)  # one gv, every attrs for it

    gv_judge = "\tif gv == " + str("'" + n[one_gv] + "'") + ":\n"
    Map_index += "+".join(indexing_cols) + "]\n"  # index = MFMap[str(tmprec['cust'])+str(tmprec['prod'])]
    Condition += " and ".join(
        conditioning_cols) + "):\n"  # if ((tmprec['cust']==MFVector[index]['cust']) and (tmprec['prod']==MFVector[index]['prod'])):
    Update += "".join(updating_cols) + "\n"  # MFVector[index]['2_count_quant']+=1
    # print(Map_index,Condition, Update)
    one_gv_scan_text = gv_judge + scan_text_2 + Map_index + Condition + Update

    MF_update += one_gv_scan_text

print(MF_update)

for gv in ['x', 'y']:
	if gv == 'x':
		for i in range(len(records)):
			tmprec = {x[0]:x[1] for x in zip(['cust', 'prod', 'day', 'month', 'year', 'state', 'quant'],records[i])}
			index = MFMap[str(tmprec['prod'])+str(tmprec['cust'])]
			if ((tmprec['prod']==MFVector[index]['prod']) and (tmprec['cust']==MFVector[index]['cust'])):
				MFVector[index]['x_count_quant']+=1
				MFVector[index]['x_sum_quant']+=tmprec['quant']
				MFVector[index]['x_avg_quant']=MFVector[index]['x_sum_quant']/MFVector[index]['x_count_quant']
				MFVector[index]['x_count_prod']+=1

	if gv == 'y':
		for i in range(len(records)):
			tmprec = {x[0]:x[1] for x in zip(['cust', 'prod', 'day', 'month', 'year', 'state', 'quant'],records[i])}
			index = MFMap[str(tmprec['cust'])]
			if ((tmprec['prod']!=MFVector[index]['prod']) and (tmprec['cust']==MFVector[index]['cust'])):
				MFVector[index]['y_count_quant']+=1
				MFVector[index]['y_sum_quant']+=tmprec['quant']
				MFVector[index]['y_avg_quant']=MFVector[index]['y_s

## Having
- two functions for handling piece in condition

In [128]:
########################################
# process piece in Condition like
# ['1_sum_quant/2',"==","3"]  ==> ["MFVector[index]['1_sum_quant']/2", '==', '3']
########################################
def compute_str_process(x):

    '''
    process the computational symbol after logic processing

    input1 = ['1_sum_quant/2',"==","3"]
    output1 = ["MFVector[index]['1_sum_quant']/2", '==', '3']

    input2 = ['2_count_prod+2', '==', '1_count_prod+2']
    output2 = ["MFVector[index]['2_count_prod']+2", '==', "MFVector[index]['1_count_prod']+2"]
    '''
    # test
    out = []
    for frac in x:  # three kinds
        
        # if gv in the frac and find and it contains compute symbol  [\+\-\/\*] or other col
        if ("_" in frac) and len(re.findall('(\w+)[\+\-\/\*]', frac)) >= 1:
            col = re.findall('(\w+)[\+\-\/\*]', frac)[0]  # extract the col
            compute_symbol = re.findall('[\+\-\/\*]', frac)
            
            suffix = frac[len(col) + 1::]  # it contains compute symbol  [\+\-\/\*] or other col
            if re.findall('[\D]+', suffix):  # not pure number --> suffix is one col in MFVector
                out.extend(["MFVector[index]['" + col + "']" + compute_symbol[0] + "MFVector[index]['" + suffix + "']"])
            else:  # pure number
                out.extend(["MFVector[index]['" + col + "']" + compute_symbol[0] + suffix])
        elif ("_" in frac):  # if only the gv in the frac
            out.extend(["MFVector[index]['" + frac + "']"])
        else:  # not col in this frac. it means it is "==" or "0" or ">=", etc
            out.append(frac)
    return "".join(out)


########################################
### Parsing for G
########################################
def G_parsing(G):
    '''
    input =  ["2_count_prod+2= 1_count_prod+2 and 1_avg_quant<>0"]  # G
    output = "MFVector[index]['2_count_prod']+2==MFVector[index]['1_count_prod']+2
            and MFVector[index]['1_avg_quant']!=0"

    '''
    Having_c = []
    logic_map = {">=": " >= ", "<=": " <= ", "=": " == ", "<>": ' != ', ">": " > ", "<": " < "}  # from left to right, it only execute once. target: insert space for separating
    for one_condition in G[0].split('and'):  # piece = ["2_count_prod+2= 1_count_prod+2" and 1_avg_quant<>0]
        piece = one_condition[:]
        
        # replace once for the logic map
        for key in logic_map.keys():  # order in the logic map matter. if find the '=' first,  the">=" maybe include
            if key in piece:  # subsitute, and break into two part, based on " " whitespace because the map contains the white space in both side
                x = [i.strip() for i in piece.replace(key, logic_map[key]).split(" ") if len(i.strip()) >= 1]
                # print(x)   # x = ['2_count_prod', '==', '1_count_prod+2']
                break  # only replace once
        
        # x = ['2_count_prod', '==', '1_count_prod+2']
        # further replace for +-*/ symbol with re and add the prefix: MFVector[index] etc
        out = compute_str_process(x)
        Having_c.append("".join(out))  # "MFVector[index]['2_count_prod']==MFVector[index]['1_count_prod']+2
    return " and ".join(Having_c)


In [129]:
########################################
# Having
########################################
Having_text = 'output_idx=0\n'
Having_text += "for index in range(len(MFVector)):\n"
Having_text += "\tif " + G_parsing(G) + ":\n"
Having_text += '\t\toutput_idx+=1\n'  # for output index
Having_text += "\t\tprint(end='{:<6}|'.format(output_idx))\n"

print(Having_text)

output_idx=0
for index in range(len(MFVector)):
	if MFVector[index]['y_avg_quant']!=0 and MFVector[index]['x_avg_quant']>=0:
		output_idx+=1
		print(end='{:<6}|'.format(output_idx))



## Selection

In [130]:
########################################
# Output
########################################
### Table_cols_name_output
Table_cols_name_output = "print('='*100)\n"
format_ = "print('{:<6}|"
cols = "'.format('index'"

for col in S:
    try:  # detect exist or not in MF_dtypes
        format_ += format_length_map[MF_dtypes[i]] + '|'
        cols += ",'" + col + "'"
    except:  # if not , it is float, and the selection part need change
        format_ += format_length_map[0] + '|'
        cols += ",'" + col + "'"
Table_cols_name_output += format_ + cols + '))\n' + "print('='*100)\n"

Selection_text = ''
for col in S:
    try:  # simple
        if type(MF_dtypes[col]) == str:
            Selection_text += "\t\tprint('{:<15}|'.format(MFVector[index]['" + col + "']), end='')\n"
        elif (type(MF_dtypes[col]) == int):
            Selection_text += "\t\tprint('{:>15.0f}|'.format(MFVector[index]['" + col + "']), end='')\n"
        elif (type(MF_dtypes[col]) == float):
            Selection_text += "\t\tprint('{:>15.2f}|'.format(MFVector[index]['" + col + "']), end='')\n"
    except:  # the combined output
        Selection_text += "\t\tprint('{:>15.2f}|'.format(" + compute_str_process(
            [col]) + "), end='')\n"  # it can only process list

Selection_text += "\t\tprint('')"

print(Selection_text)

		print('{:<15}|'.format(MFVector[index]['prod']), end='')
		print('{:<15}|'.format(MFVector[index]['cust']), end='')
		print('{:>15.2f}|'.format(MFVector[index]['x_sum_quant']), end='')
		print('{:>15.0f}|'.format(MFVector[index]['x_count_prod']), end='')
		print('{:>15.2f}|'.format(MFVector[index]['x_avg_quant']+1), end='')
		print('{:>15.2f}|'.format(MFVector[index]['y_avg_quant']/2), end='')
		print('{:>15.2f}|'.format(MFVector[index]['x_avg_quant']/MFVector[index]['y_avg_quant']), end='')
		print('')


## Combined the result and return

In [131]:
res = '# Import and Connecting DB\n'
res += import_text + '\n'
res += connection_text + '\n'
res += '# Initializing MF_Vector\n'
res += MFVector_text + '\n'
res += '# Initializing Map for MF_Structure\n'
res += MFMap_text + '\n'
res += '# Updating for MF_Structure\n'
res += MF_update + '\n'
res += '# Outputing\n'
res += Table_cols_name_output
res += '# Having\n'
res += Having_text + '\n'
res += Selection_text + '\n'

In [132]:
########################################
# Save result
########################################
to_file = "./Output_files/out%s.py" % output_num
with open(to_file, "w", encoding='utf-8') as f:
    f.write(res)
    print("The file has been saved: ", to_file)

The file has been saved:  ./Output_files/out7_2.py


## Preview of output

In [133]:
print(res)

# Import and Connecting DB
import psycopg2
import sys
from collections import defaultdict

try:
    connection = psycopg2.connect(user='postgres', password='00000000', host='localhost', port='5388', database='postgres', )
    query = "SELECT * FROM sales"
    cursor = connection.cursor()
    cursor.execute(query)
    if cursor.rowcount == 0:
        print("Connected, but no data!")
        sys.exit(0)
                    
    records = cursor.fetchall()
except(Exception, psycopg2.Error) as error:
    print("Error connecting to PostgreSQL database ==>", error)
    sys.exit(0)

                    
# Initializing MF_Vector
MFVector = defaultdict(lambda: {'prod':'', 'cust':'', 'x_count_quant':0, 'x_sum_quant':0.0, 'x_avg_quant':0.0, 'y_count_quant':0, 'y_sum_quant':0.0, 'y_avg_quant':0.0, 'x_count_prod':0, 'x_sum_quant':0.0})
MF_idx=0
MFMap = {}

# Initializing Map for MF_Structure
for i in range(len(records)):
	tmprec = {x[0]:x[1] for x in zip(['cust', 'prod', 'day', 'month', 'year', 'st

In [134]:
# !python ./Output_files/out9.py

# Py excute Result - <font color=red>Need copy back</font>

In [137]:
def test():
    # Import and Connecting DB
    import psycopg2
    import sys
    from collections import defaultdict

    try:
        connection = psycopg2.connect(user='postgres', password='00000000', host='localhost', port='5388', database='postgres', )
        query = "SELECT * FROM sales"
        cursor = connection.cursor()
        cursor.execute(query)
        if cursor.rowcount == 0:
            print("Connected, but no data!")
            sys.exit(0)

        records = cursor.fetchall()
    except(Exception, psycopg2.Error) as error:
        print("Error connecting to PostgreSQL database ==>", error)
        sys.exit(0)


    # Initializing MF_Vector
    MFVector = defaultdict(lambda: {'prod':'', 'cust':'', 'x_count_quant':0, 'x_sum_quant':0.0, 'x_avg_quant':0.0, 'y_count_quant':0, 'y_sum_quant':0.0, 'y_avg_quant':0.0, 'x_count_prod':0, 'x_sum_quant':0.0})
    MF_idx=0
    MFMap = {}

    # Initializing Map for MF_Structure
    for i in range(len(records)):
        tmprec = {x[0]:x[1] for x in zip(['cust', 'prod', 'day', 'month', 'year', 'state', 'quant'],records[i])}
        if (str(tmprec['prod']) + str(tmprec['cust'])) not in MFMap:
            MFVector[MF_idx]['prod']=tmprec['prod']
            MFVector[MF_idx]['cust']=tmprec['cust']
            MFMap[str(tmprec['prod'])]=MF_idx
            MFMap[str(tmprec['prod']) + str(tmprec['cust'])]=MF_idx
            MFMap[str(tmprec['cust']) + str(tmprec['prod'])]=MF_idx
            MFMap[str(tmprec['cust'])]=MF_idx
            MF_idx+=1 # new idx


    # Updating for MF_Structure
    for gv in ['x', 'y']:
        if gv == 'x':
            for i in range(len(records)):
                tmprec = {x[0]:x[1] for x in zip(['cust', 'prod', 'day', 'month', 'year', 'state', 'quant'],records[i])}
                index = MFMap[str(tmprec['cust'])+str(tmprec['prod'])]
                if ((tmprec['cust']==MFVector[index]['cust']) and (tmprec['prod']==MFVector[index]['prod']) and (tmprec['prod']==MFVector[index]['prod'])):
                    MFVector[index]['x_count_prod']+=1
                    MFVector[index]['x_count_quant']+=1
                    MFVector[index]['x_sum_quant']+=tmprec['quant']
                    MFVector[index]['x_avg_quant']=MFVector[index]['x_sum_quant']/MFVector[index]['x_count_quant']

        if gv == 'y':
            for i in range(len(records)):
                tmprec = {x[0]:x[1] for x in zip(['cust', 'prod', 'day', 'month', 'year', 'state', 'quant'],records[i])}
                index = MFMap[str(tmprec['cust'])]
                if ((tmprec['cust']==MFVector[index]['cust']) and (tmprec['prod']!=MFVector[index]['prod'])):
                    MFVector[index]['y_count_quant']+=1
                    MFVector[index]['y_sum_quant']+=tmprec['quant']
                    MFVector[index]['y_avg_quant']=MFVector[index]['y_sum_quant']/MFVector[index]['y_count_quant']


    # Outputing
    print('='*100)
    print('{:<6}|{:>15}|{:>15}|{:>15}|{:>15}|{:>15}|{:>15}|{:>15}|'.format('index','prod','cust','x_sum_quant','x_count_prod','x_avg_quant+1','y_avg_quant/2','x_avg_quant/y_avg_quant'))
    print('='*100)
    # Having
    output_idx=0
    for index in range(len(MFVector)):
        if MFVector[index]['y_avg_quant']!=0 and MFVector[index]['x_avg_quant']>=0:
            output_idx+=1
            print(end='{:<6}|'.format(output_idx))

            print('{:<15}|'.format(MFVector[index]['prod']), end='')
            print('{:<15}|'.format(MFVector[index]['cust']), end='')
            print('{:>15.2f}|'.format(MFVector[index]['x_sum_quant']), end='')
            print('{:>15.0f}|'.format(MFVector[index]['x_count_prod']), end='')
            print('{:>15.2f}|'.format(MFVector[index]['x_avg_quant']+1), end='')
            print('{:>15.2f}|'.format(MFVector[index]['y_avg_quant']/2), end='')
            print('{:>15.2f}|'.format(MFVector[index]['x_avg_quant']/MFVector[index]['y_avg_quant']), end='')
            print('')


            
    return MFVector


In [140]:
import pandas as pd
MFVector_result = pd.DataFrame(test()).T
MFVector_result

index |           prod|           cust|    x_sum_quant|   x_count_prod|  x_avg_quant+1|  y_avg_quant/2|x_avg_quant/y_avg_quant|
1     |Bread          |Helen          |       25032.00|              9|        2782.33|        1129.56|           1.23|
2     |Butter         |Knuth          |        7545.00|              6|        1258.50|        1184.28|           0.53|
3     |Coke           |Sam            |       11354.00|              5|        2271.80|        1195.95|           0.95|
4     |Coke           |Emily          |       21681.00|              9|        2410.00|        1261.04|           0.96|
5     |Bread          |Bloom          |       25343.00|             12|        2112.92|        1253.29|           0.84|


,prod,cust,x_count_quant,x_sum_quant,x_avg_quant,y_count_quant,y_sum_quant,y_avg_quant,x_count_prod
0,Pepsi,Bloom,8,15390,1923.75,0,0,0,8
1,Bread,Knuth,12,32562,2713.5,0,0,0,12
2,Pepsi,Emily,7,22782,3254.57,0,0,0,7
3,Fruits,Emily,13,35059,2696.85,0,0,0,13
4,Milk,Helen,12,25239,2103.25,0,0,0,12
5,Soap,Emily,8,19290,2411.25,0,0,0,8
6,Eggs,Bloom,11,27553,2504.82,0,0,0,11
7,Yogurt,Bloom,8,17670,2208.75,0,0,0,8
8,Pepsi,Helen,13,34613,2662.54,0,0,0,13
9,Bread,Emily,10,15343,1534.3,0,0,0,10


In [139]:
result.describe()

,prod,cust,x_count_quant,x_sum_quant,x_avg_quant,y_count_quant,y_sum_quant,y_avg_quant,x_count_prod
count,50,50,50,50.0,50.0,50,50.0,50.0,50
unique,10,5,16,50.0,50.0,6,6.0,6.0,16
top,Eggs,Sam,8,25343.0,1534.3,0,0.0,0.0,8
freq,5,10,9,1.0,1.0,45,45.0,45.0,9
